# interaction_builder

In [ ]:
#| default_exp interaction_builder

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from tk_slack.core import ValueFormatter
from tk_slack.slack_actions import ActionIdManager

from fastcore.basics import patch_to
from fastcore.test import *

from typing import List, Tuple, Dict, Any, Callable, Optional

import pandas as pd
import json

# InteractionBuilder

In [ ]:
#| export

class InteractionBuilder:
    """
    Utility class for creating interactive Slack Block Kit elements.
    """
    pass

First, let's start at the top of the message, which would be the message's header:

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_button(self, text: str, action_id: str, url: Optional[str] = None, 
                    value: Optional[str] = None, style: Optional[str] = None) -> Dict[str, Any]:
    """Create a button element for Slack messages.
    
    Args:
        text: Button text
        action_id: Identifier for the button
        url: Optional URL for link buttons
        value: Optional value for the button
        style: Optional style - "primary" or "danger"
        
    Returns:
        Slack block kit button object
    """
    button = {
        "type": "button",
        "text": {
            "type": "plain_text",
            "text": text,
            "emoji": True
        },
        "action_id": action_id
    }
    
    if url:
        button["url"] = url
        
    if value:
        button["value"] = value
        
    if style and style in ["primary", "danger"]:
        button["style"] = style
        
    return button

In [ ]:
test_eq(InteractionBuilder.create_button('My Button','button_clicked'),
        {'type': 'button',
        'text': {'type': 'plain_text', 'text': 'My Button', 'emoji': True},
        'action_id': 'button_clicked'})

InteractionBuilder.create_button('My Button','button_clicked')

{'type': 'button',
 'text': {'type': 'plain_text', 'text': 'My Button', 'emoji': True},
 'action_id': 'button_clicked'}

We also need a way to turn the action elements into the full action section/block:

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_actions_block(self, elements: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Create an actions block for Slack messages.
    
    Args:
        elements: List of interactive elements
        
    Returns:
        Slack block kit actions object
    """
    return {"type": "actions","elements": elements}

In [ ]:
test_eq(
    InteractionBuilder.create_actions_block(
        InteractionBuilder.create_button('Another Button','button_2_clicked')),
    {'type': 'actions',
    'elements': {'type': 'button',
    'text': {'type': 'plain_text', 'text': 'Another Button', 'emoji': True},
    'action_id': 'button_2_clicked'}}
)


InteractionBuilder.create_actions_block(
    InteractionBuilder.create_button('Another Button','button_2_clicked')
)

{'type': 'actions',
 'elements': {'type': 'button',
  'text': {'type': 'plain_text', 'text': 'Another Button', 'emoji': True},
  'action_id': 'button_2_clicked'}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_datepicker(self, action_id: str, placeholder: str, 
                         initial_date: Optional[str] = None) -> Dict[str, Any]:
        """Create a date picker element.
        
        Args:
            action_id: Identifier for the date picker
            placeholder: Placeholder text
            initial_date: Optional initial date (YYYY-MM-DD format)
            
        Returns:
            Slack block kit datepicker object
        """
        datepicker = {
            "type": "datepicker",
            "action_id": action_id,
            "placeholder": {
                "type": "plain_text",
                "text": placeholder,
                "emoji": True
            }
        }
        
        if initial_date:
            datepicker["initial_date"] = initial_date
            
        return datepicker

In [ ]:
test_eq(InteractionBuilder.create_datepicker('start_date','Pick a Start Date'),
        {'type': 'datepicker',
            'action_id': 'start_date',
            'placeholder': {'type': 'plain_text',
            'text': 'Pick a Start Date',
            'emoji': True}})

InteractionBuilder.create_datepicker('start_date','Pick a Start Date')

{'type': 'datepicker',
 'action_id': 'start_date',
 'placeholder': {'type': 'plain_text',
  'text': 'Pick a Start Date',
  'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_static_select(self, action_id: str, placeholder: str, 
                        options: List[Tuple[str, str]]) -> Dict[str, Any]:
    """Create a dropdown select element.
    
    Args:
        action_id: Identifier for the select
        placeholder: Placeholder text
        options: List of (text, value) tuples for options
        
    Returns:
        Slack block kit static_select object
    """
    select = {
        "type": "static_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        },
        "options": [
            {
                "text": {
                    "type": "plain_text",
                    "text": text,
                    "emoji": True
                },
                "value": value
            }
            for text, value in options
        ]
    }
    
    return select

In [ ]:
test_eq(InteractionBuilder.create_static_select('credit_action','Choose and Action',[('Issue Creidt','issue_credit'),('Ignore','ignore')]),
        {'type': 'static_select',
            'action_id': 'credit_action',
            'placeholder': {'type': 'plain_text',
            'text': 'Choose and Action',
            'emoji': True},
            'options': [{'text': {'type': 'plain_text',
                'text': 'Issue Creidt',
                'emoji': True},
            'value': 'issue_credit'},
            {'text': {'type': 'plain_text', 'text': 'Ignore', 'emoji': True},
            'value': 'ignore'}]})

InteractionBuilder.create_static_select('credit_action','Choose and Action',[('Issue Creidt','issue_credit'),('Ignore','ignore')])

{'type': 'static_select',
 'action_id': 'credit_action',
 'placeholder': {'type': 'plain_text',
  'text': 'Choose and Action',
  'emoji': True},
 'options': [{'text': {'type': 'plain_text',
    'text': 'Issue Creidt',
    'emoji': True},
   'value': 'issue_credit'},
  {'text': {'type': 'plain_text', 'text': 'Ignore', 'emoji': True},
   'value': 'ignore'}]}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_multi_select(self, action_id: str, placeholder: str, 
                        options: List[Tuple[str, str]]) -> Dict[str, Any]:
    """Create a multi-select dropdown element.
    
    Args:
        action_id: Identifier for the multi-select
        placeholder: Placeholder text
        options: List of (text, value) tuples for options
        
    Returns:
        Slack block kit multi_static_select object
    """
    multi_select = {
        "type": "multi_static_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        },
        "options": [
            {
                "text": {
                    "type": "plain_text",
                    "text": text,
                    "emoji": True
                },
                "value": value
            }
            for text, value in options
        ]
    }
    
    return multi_select

In [ ]:
test_eq(InteractionBuilder.create_multi_select('select_multiple','Select Option(s)',[('One','one'),('Two','two')]),
        {'type': 'multi_static_select',
            'action_id': 'select_multiple',
            'placeholder': {'type': 'plain_text',
            'text': 'Select Option(s)',
            'emoji': True},
            'options': [{'text': {'type': 'plain_text', 'text': 'One', 'emoji': True},
            'value': 'one'},
            {'text': {'type': 'plain_text', 'text': 'Two', 'emoji': True},
            'value': 'two'}]})

InteractionBuilder.create_multi_select('select_multiple','Select Option(s)',[('One','one'),('Two','two')])

{'type': 'multi_static_select',
 'action_id': 'select_multiple',
 'placeholder': {'type': 'plain_text',
  'text': 'Select Option(s)',
  'emoji': True},
 'options': [{'text': {'type': 'plain_text', 'text': 'One', 'emoji': True},
   'value': 'one'},
  {'text': {'type': 'plain_text', 'text': 'Two', 'emoji': True},
   'value': 'two'}]}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_users_select(self, action_id: str, placeholder: str) -> Dict[str, Any]:
        """Create a user select element.
        
        Args:
            action_id: Identifier for the users select
            placeholder: Placeholder text
            
        Returns:
            Slack block kit users_select object
        """
        return {
            "type": "users_select",
            "action_id": action_id,
            "placeholder": {
                "type": "plain_text",
                "text": placeholder,
                "emoji": True
            }
        }

In [ ]:
test_eq(InteractionBuilder.create_users_select('selected_user','Select a User'),
        {'type': 'users_select',
        'action_id': 'selected_user',
        'placeholder': {'type': 'plain_text', 'text': 'Select a User', 'emoji': True}})

InteractionBuilder.create_users_select('selected_user','Select a User')

{'type': 'users_select',
 'action_id': 'selected_user',
 'placeholder': {'type': 'plain_text', 'text': 'Select a User', 'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def create_channels_select(self, action_id: str, placeholder: str) -> Dict[str, Any]:
    """Create a channel select element.
    
    Args:
        action_id: Identifier for the channels select
        placeholder: Placeholder text
        
    Returns:
        Slack block kit channels_select object
    """
    return {
        "type": "channels_select",
        "action_id": action_id,
        "placeholder": {
            "type": "plain_text",
            "text": placeholder,
            "emoji": True
        }
    }

In [ ]:
test_eq(InteractionBuilder.create_channels_select('selected_channel','Select a Channel'),
        {'type': 'channels_select',
        'action_id': 'selected_channel',
        'placeholder': {'type': 'plain_text',
        'text': 'Select a Channel',
        'emoji': True}})

InteractionBuilder.create_channels_select('selected_channel','Select a Channel')

{'type': 'channels_select',
 'action_id': 'selected_channel',
 'placeholder': {'type': 'plain_text',
  'text': 'Select a Channel',
  'emoji': True}}

In [ ]:
#| export

@patch_to(InteractionBuilder,cls_method=True)
def detect_and_create_interactive_elements(
        self,
        option_names: List[str], 
        option_values: List[str], 
        action_type: Optional[str] = None,
        metadata: Optional[str] = None,
        view_info: Optional[Dict[str, Any]] = None
        ) -> List[Dict[str, Any]]:
    """Smartly detect and create appropriate interactive elements.
    
    Args:
        option_names: List of option names
        option_values: List of option values
        action_type: Optional explicit action type
        metadata: Optional metadata to include in action_id
        view_info: Optional view information for embedding in metadata
        
    Returns:
        List of interactive elements
    """
    
    # Smart action type detection if not explicitly set
    if not action_type:
        # Check for date-related names to use date picker
        date_keywords = ['date', 'when', 'calendar', 'schedule', 'deadline', 'due']
        has_date_keyword = any(keyword in ' '.join(option_names).lower() for keyword in date_keywords)
        
        # Check if we have multiple options for a select
        if len(option_names) > 3:
            action_type = 'static_select'
        elif has_date_keyword:
            action_type = 'datepicker'
        elif any('user' in name.lower() for name in option_names):
            action_type = 'users_select'
        elif any('channel' in name.lower() for name in option_names):
            action_type = 'channels_select'
        elif any(['multi' in ''.join(option_names).lower(), 
                    'select multiple' in ''.join(option_names).lower()]):
            action_type = 'multi_static_select'
        else:
            action_type = 'button'
    
    # If view_info is provided, embed it in the metadata
    if view_info and metadata:
        # Combine view_info with existing metadata
        try:
            meta_dict = json.loads(metadata) if isinstance(metadata, str) else metadata
            if isinstance(meta_dict, dict) and isinstance(view_info, dict):
                meta_dict.update(view_info)
                metadata = json.dumps(meta_dict)
        except:
            # If metadata isn't JSON, append view info to metadata
            view_info_str = json.dumps(view_info)
            metadata = f"{metadata}|{view_info_str}"
    elif view_info:
        # Just use view_info as metadata
        metadata = json.dumps(view_info)
    
    elements = []
    
    # Create appropriate interactive elements based on action_type
    if action_type == 'datepicker':
        # Create date picker
        initial_date = None
        if option_values and option_values[0]:
            try:
                # Try to parse as date if it looks like one
                parsed_date = pd.to_datetime(option_values[0])
                initial_date = parsed_date.strftime('%Y-%m-%d')
            except Exception:
                pass
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0
            )
        
        elements.append(
            InteractionBuilder.create_datepicker(
                action_id=action_id,
                placeholder=option_names[0],
                initial_date=initial_date
            )
        )
        
    elif action_type == 'static_select':
        # Create dropdown with options
        options = [(name, value) for name, value in zip(option_names, option_values)]
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0
        )
        
        elements.append(
            InteractionBuilder.create_static_select(
                action_id=action_id,
                placeholder="Select an option",
                options=options
            )
        )
        
    elif action_type == 'multi_static_select':
        # Create multi-select dropdown
        options = [(name, value) for name, value in zip(option_names, option_values)]
        
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_multi_select(
                action_id=action_id,
                placeholder="Select options",
                options=options
            )
        )
        
    elif action_type == 'users_select':
        # Create user select
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0
        )
        
        elements.append(
            InteractionBuilder.create_users_select(
                action_id=action_id,
                placeholder=option_names[0] if option_names else "Select a user"
            )
        )
        
    elif action_type == 'channels_select':
        # Create channel select
        # Generate unique action_id with metadata
        action_id = ActionIdManager.generate_action_id(
            action_type=action_type,
            index=0,
            metadata=metadata
        )
        
        elements.append(
            InteractionBuilder.create_channels_select(
                action_id=action_id,
                placeholder=option_names[0] if option_names else "Select a channel"
            )
        )
        
    else:
        # Default to buttons
        for i, (name, value) in enumerate(zip(option_names, option_values)):
            # Generate unique action_id with metadata
            action_id = ActionIdManager.generate_action_id(
                action_type=action_type,
                index=i,
            )
            
            elements.append(
                InteractionBuilder.create_button(
                    text=str(name),
                    action_id=action_id,
                    value=str(value),
                    style="primary" if i == 0 else None
                )
            )
    
    return elements

In [ ]:
test_eq(InteractionBuilder.detect_and_create_interactive_elements(['Start Date'],['start_date'],'selected_start_date'),
        [{'type': 'button',
            'text': {'type': 'plain_text', 'text': 'Start Date', 'emoji': True},
            'action_id': 'tk_interaction_sele_0',
            'value': 'start_date',
            'style': 'primary'}])

InteractionBuilder.detect_and_create_interactive_elements(['Start Date'],['start_date'],'selected_start_date')

[{'type': 'button',
  'text': {'type': 'plain_text', 'text': 'Start Date', 'emoji': True},
  'action_id': 'tk_interaction_sele_0',
  'value': 'start_date',
  'style': 'primary'}]

#| hide

# Export
This is mainly here just to allow me to fold sections above here and still have this section visible

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()